In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as st
from scipy.stats import chi2_contingency
from scipy.stats import shapiro 

import statsmodels.formula.api as smf

In [2]:
#Загрузим данные, посмотрим на содержимое
dating_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-pozdniakov/statistic/dating_data.csv')
dating_data.head()

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1


In [7]:
zero_group = dating_data.query('group == 0')
first_group = dating_data.query('group == 1')
print('размер выборки 0-группы =', zero_group['group'].count())
print('размер выборки 1-группы =', first_group['group'].count())
print('среднее 0-группы =', zero_group['is_match'].mean())
print('среднее 1-группы =', first_group['is_match'].mean())

print('Доверительный интервал 0-группы =', st.t.interval(alpha=0.95, df=len(zero_group['is_match'])-1, 
                                                         loc=np.mean(zero_group['is_match']), 
                                                         scale=st.sem(zero_group['is_match']))) 
print('Доверительный интервал 1-группы =', st.t.interval(alpha=0.95, df=len(first_group['is_match'])-1, 
                                                         loc=np.mean(first_group['is_match']), 
                                                         scale=st.sem(first_group['is_match'])))
print(shapiro(zero_group.is_match))
print(shapiro(first_group.is_match))
#sns.displot(dating_data, x='is_match', hue='group')

размер выборки 0-группы = 4792
размер выборки 1-группы = 9722
среднее 0-группы = 0.19490818030050083
среднее 1-группы = 0.40207776177741206
Доверительный интервал 0-группы = (0.18368844847587973, 0.20612791212512194)
Доверительный интервал 1-группы = (0.3923295697244652, 0.4118259538303589)
ShapiroResult(statistic=0.48412472009658813, pvalue=0.0)
ShapiroResult(statistic=0.622529149055481, pvalue=0.0)


/opt/tljh/user/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


У нас есть две группы: 0 группа - со старым алгоритмом подбора, потенциального партнера, и 1 группа - с новым алгоритмом.

Примем что идеальный алгоритм - когда match-исход между двумя подобранными друг для друга пользователями случается в 100% 
случаях, Тогда за конверсию в наших данных можно принять долю положительных match-событий

Размер выборки первой группы практически в 2 раза больше чем размер нулевой группы.
Среднее первой группы более чем в два раза превышает среднее нулевой группы,
а доверительные интервалы выборок с заданной надежностью 95% не пересекаются.

На первый взгляд можно сделать очевидный вывод что новый алгоритм значительно улучшает конверсию, это следует из средних
по группам

Учитывая что в нашем случае is_match принимает всего два значения,то есть является 
категориальной переменной - для A/B тестирования будем использовать критерий согласия Пирсона(Хи-квадрат).

Сформулируем гипотезы, которые будем тестировать:

Нулевая гипотеза - распределение(в нашем случае конверсия match-событий) не зависит от группы тестирования
Альтернативная гипотеза - распределение match-событий зависит от групп тестирования

Для начала анализа основной нашей задачей является преобразование исходных данных в таблицу сопряженности,
содержащую номера групп и результатов is_match с разбивкой по количеству удачных и неудачных исходов

Наши данные не требуют кодирования фиктивными перемеменными, потому что у нас уже имеется необходимое для анализа поле
group, которое содержит всего две группы и удобный для анализа формат(0 и 1)

In [4]:
cross_data_from_chi = dating_data.groupby(['group','is_match'], as_index = False) \
                                .agg({'user_id_1':'count'}) \
                                .rename(columns={'user_id_1':'count_events'}) \
                                .pivot_table(index='group', columns='is_match', values='count_events')
cross_data_from_chi = cross_data_from_chi.reset_index()[[0,1]].to_numpy()

In [5]:
res_statistic, res_pvalue, res_dof, res_expected_freq = chi2_contingency(cross_data_from_chi)
print('T-statistic =', res_statistic)
print('p-value =', res_pvalue)
print('The degrees of freedom =', res_dof)

T-statistic = 618.6889141576198
p-value = 1.4418299163662586e-136
The degrees of freedom = 1


In [ ]:
По результатам проведенного теста хи-квадрат мы однозначно можем отклонить нулевую гипотезу и сделать вывод:
    Конверсия зависит от групп тестирования
    
Далее мы можем проверить на сколько сильно влияет новый алгоритм на конверсию для наших данных,
воспользуемся регрессионным анлизом из библиотеки statsmodels.
Зададим в качестве зависимой переменной is_match, а в качестве независимой - group

In [6]:
results = smf.ols('is_match ~ group', dating_data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               is_match   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     647.2
Date:                Mon, 22 May 2023   Prob (F-statistic):          1.04e-139
Time:                        22:36:22   Log-Likelihood:                -9366.5
No. Observations:               14514   AIC:                         1.874e+04
Df Residuals:                   14512   BIC:                         1.875e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1949      0.007     29.243      0.0

Вывод: Исходя из результатов регрессионного анализа мы видем что в среднем при применении старого алгоритма 
       Intercept == 0.1949, то есть вероятность match-исхода около 20%
       При применении нового алгоритма(в результатах выше - group), вероятность match-исхода увеличивается приблизительно на 
       21%, то есть вырастает вдвое.
       Соответственно рекомендуется безоговорочно выкатить новый алгоритм подбора партнеров на всех пользователей! 